# Feature engineering
Feature engineering is used to create extra variables form the exisiting psychological signals to have extra training data. This is only applied ot the `physio_trans_data_session.pickle`, because those are the independent variables for the ML models.

In [2]:
# import dependencies
import pickle
import os
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

BASE_DIR = os.getcwd()  # Works in Jupyter
print(BASE_DIR)

# specify paths
questionnaire_path = "../data/raw/2_Questionnaire/Transformed/quest_trans_data_segments.pickle"
physio_path = "../data/raw/3_Physio/Transformed/physio_trans_data_segments.pickle"
annotation_path = "../data/raw/4_Annotation/Transformed/ann_trans_data_segments.pickle"


c:\Users\Zita\Repositories\affective-states\notebooks


# Original dataset

In [3]:
with open(physio_path, 'rb') as f:
    data = pickle.load(f)

# Inspect the type and structure
print(type(data))  # Check the data type

<class 'dict'>


In [4]:
data.keys()

dict_keys(['filt_EDA', 'filt_PPG', 'ts', 'sampling_rate', 'packet_number', 'EDR', 'hr', 'raw_EDA', 'raw_PPG', 'hr_idx', 'EDA_quality_idx', 'PPG_quality_idx'])

## Dictionary keys explanation
**filt_EDA** : Filtered signal of Electro Dermal Activity, which assesses the naturally occurring changes in electrical properties of human skin, measure sweat gland activity.

**filt_PPG** : The photoplethysmographic (PPG) signal is defined as oscillations in light transmission through a tissue.  It provides a continuous signal that can be analyzed to derive different cardiovascular metrics, including heart rate.

**ts** : Timestamps in seconds.

**packet_number** : Unknown and not relevant.

**raw_EDA** : Raw signal of EDA. (not used)

**sampling_rate** : The amount of samples per second. It is 100 for all segments, which means that the time between each sample is 0.1 seconds, as you can see by the calculation in the cell below.

**raw_PPG** : Raw singal of PPG. (not used)

**hr** : Heart rate in bpm (beats per minutes.

**EDR** : Electro Dermal Response which is the derivative of EDA.

**hr_idx** : Derivate of heart rate.

**EDA_quality_idx** : Quality index of the EDA signal defined by a float between 0 and 240.

**PPG_quality_idx** : Quality index of the PPG singal defined by a float between 0 and 240.

This dataset is sourced from Boda et al. (2024) and bad quality data has already been discarded and the dataset is cleaned. There can be missing values and the segments have different lenghts. We will use feature engineering to add more features to this dataset.

In [5]:
print(data["sampling_rate"])

[np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(100), np.int64(

In [6]:
# calculate sampling rate

sampling_rate = data["sampling_rate"][0]  # seconds^-1
dt = 1.0 / sampling_rate
print(dt) # this matches what we got above

0.01


In [7]:
print("Number of instances: " + str(len(data['filt_EDA']) ))

Number of instances: 1481


In [8]:
print(data['filt_EDA'][1])

print("\n Example of filtered EDA signal length (of instance 1): " +  str(len(data['filt_EDA'][1])))

[0. 0. 0. ... 0. 0. 0.]

 Example of filtered EDA signal length (of instance 1): 2000


# Feature engineering
The following feature are added:

**SCL** : 

**SCR** : Phasic EDA signal.

**hr** : Heart rate in bpm

**HRV** : Heart Rate Variability

**EDA peaks**:


TODO: finish the definitions for all the new features

In [9]:
# make data into a df_physio_clean_clean dataframe
df_physio = pd.DataFrame(dict([ (key, pd.Series(val)) for key, val in data.items() ]))

# Display the resulting DataFrame
df_physio.head()

,filt_EDA,filt_PPG,ts,sampling_rate,packet_number,EDR,hr,raw_EDA,raw_PPG,hr_idx,EDA_quality_idx,PPG_quality_idx
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-600.060815151149, -595.2853451896902, -587.9...","[5.0, 5.01, 5.0200000000000005, 5.03, 5.04, 5....",100,"[8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[71.2523979172376, 68.60128444958119, 66.18906...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1191.0, 1193.0, 1200.0, 1204.0, 1206.0, 1216....","[187, 276, 368, 459, 542, 625, 707, 788, 866, ...",5.0,0.0
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1627.5018611126122, 1609.5223131853395, 1600....","[621.0, 621.01, 621.02, 621.03, 621.04, 621.05...",100,"[8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[70.66836951894423, 70.15177388417735, 71.8546...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3535.0, 3533.0, 3534.0, 3536.0, 3534.0, 3536....","[173, 254, 343, 424, 508, 611, 713, 813, 910, ...",6.0,1.0
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1483.4034056640082, 1513.4885977757735, 1543....","[467.0, 467.01, 467.02, 467.03000000000003, 46...",100,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[83.51101634683724, 74.71290165712789, 71.1779...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3536.0, 3536.0, 3536.0, 3536.0, 3536.0, 3539....","[150, 234, 329, 405, 493, 580, 666, 758, 829, ...",8.0,2.0
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-876.333693217288, -879.9930179232609, -878.3...","[313.0, 313.01, 313.02, 313.03000000000003, 31...",100,"[8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[66.20553359683794, 73.45191040843214, 71.9148...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[607.0, 595.0, 594.0, 601.0, 608.0, 624.0, 643...","[126, 214, 283, 382, 470, 547, 648, 724, 811, ...",9.0,3.0
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1236.631858494471, -1220.5943338457344, -119...","[159.0, 159.01, 159.02, 159.03, 159.04, 159.05...",100,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[82.36125839519265, 76.63156510230421, 71.7424...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[442.0, 448.0, 465.0, 479.0, 502.0, 533.0, 560...","[106, 188, 274, 357, 427, 510, 593, 672, 759, ...",10.0,4.0


In [10]:
def arr_calculation(df, col, type, name=None):
    if name is None:
        name = type + "_" + col

    if type == "mean":
        df[name] = np.array([np.mean(arr) if len(arr) > 0 else np.nan for arr in df[col]])
    elif type == "std":
        df[name] = np.array([np.std(arr) if len(arr) > 0 else np.nan for arr in df[col]])
    elif type == "max":     
        df[name] = np.array([np.max(arr) if len(arr) > 0 else np.nan for arr in df[col]])
    elif type == "min":
        df[name] = np.array([np.min(arr) if len(arr) > 0 else np.nan for arr in df[col]])  
    elif type == "gradient":
        df[name] = [np.gradient(arr) if len(arr) > 0 else [] for arr in df_physio[col]] 
    elif type == "diff":
        df[name] = [np.diff(arr) if len(arr) > 0 else [] for arr in df_physio[col]]

    return df


def statistics(df, col):
    df = arr_calculation(df, col, "mean")
    df = arr_calculation(df, col, "std")
    df = arr_calculation(df, col, "max")
    df = arr_calculation(df, col, "min")
    return df

In [11]:
# add summary statistics on EDA to the dataframe
df_physio_clean_clean_clean = statistics(df_physio, 'filt_EDA')
# compute the first derivate on EDA and add summary statistics on filt_EDA_dot to the dataframe
df_physio = arr_calculation(df_physio, 'filt_EDA', 'gradient', 'filt_EDA_dot')
df_physio = statistics(df_physio, 'filt_EDA_dot')
# compute the second derivative on EDA and add summary statistics on filt_EDA_dott to the dataframe
df_physio = arr_calculation(df_physio, 'filt_EDA_dot', 'gradient', 'filt_EDA_ddot')
df_physio = statistics(df_physio, 'filt_EDA_ddot')

In [12]:
# add summary statistics on EDR to the dataframe
df_physio = statistics(df_physio, 'EDR')
# compute the first derivate on EDR and add summary statistics on EDR_dot to the dataframe
df_physio = arr_calculation(df_physio, 'EDR', 'gradient', 'EDR_dot')
df_physio = statistics(df_physio, 'EDR_dot')
# compute the second derivative on EDR and add summary statistics on EDR_dott to the dataframe
df_physio = arr_calculation(df_physio, 'EDR_dot', 'gradient', 'EDR_ddot')
df_physio = statistics(df_physio, 'EDR_ddot')

In [13]:
# Create a time vector for the HR values (if hr_idx are valid indices into time_segment)
df_physio['hr_time'] = df_physio.apply(
    lambda row: row['ts'][row['hr_idx']] if len(row['hr_idx']) > 0 else [],     # copies the time stamps form ts to hr_time if there is a valid hr_idx
    axis=1
)

df_physio = statistics(df_physio, 'hr')

# compute the first derivative on HR and add summary statistics on hr_dot to the dataframe
df_physio['hr'] = df_physio['hr'].apply(lambda x: x if len(x) > 1 else [np.nan, np.nan])
df_physio = arr_calculation(df_physio, 'hr', 'gradient', 'hr_dot')

df_physio = statistics(df_physio, 'hr_dot')



In [14]:
# adding features on RR & HRV

df_physio = arr_calculation(df_physio, 'hr_time', 'diff')                                       # time in seconds between heart rate measurements
df_physio = statistics(df_physio, 'diff_hr_time')
df_physio = arr_calculation(df_physio, 'diff_hr_time', 'diff', 'successive_diff_hr_time')       # first derivative of diff_hr_time

bm_rr_int_count = [len(arr) > 1 for arr in df_physio["diff_hr_time"]]                           # count the number of RR intervals in each segment
                                                                                                # and check if there are at least 2 intervals    
df_physio["SDNN"] = [
    np.std(arr) * 1000 if valid else np.nan
    for arr, valid in zip(df_physio["diff_hr_time"], bm_rr_int_count)
] 

df_physio["rMSSD"] = [
    np.sqrt(np.mean(arr**2)) * 1000 if valid else np.nan
    for arr, valid in zip(df_physio["successive_diff_hr_time"], bm_rr_int_count)
]

In [15]:
peaks, properties = find_peaks(df_physio["filt_EDA"][1], height=0.1)
print(np.mean(df_physio["filt_EDA"][1][peaks]))
print(np.mean(properties["peak_heights"]))

nan
nan


c:\Users\Zita\Repositories\affective-states\venv\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Zita\Repositories\affective-states\venv\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [16]:
# adding features on EDA peaks

peak_counts, mean_peak_amp, std_peak_amp = [], [], []

for i in range(len(df_physio)):
    if len(df_physio["filt_EDA"][i]) > 0:
        peaks, properties = find_peaks(df_physio["filt_EDA"][i], height=0.1)
        peak_amplitudes = df_physio["filt_EDA"][i][peaks]
        peak_counts.append(int(len(peaks)))
        mean_peak_amp.append(np.mean(properties["peak_heights"]))
        std_peak_amp.append(np.std(properties["peak_heights"]))

df_physio[["n_peaks_EDA", "mean_peak_amp_EDA", "std_peak_amp_EDA"]] = pd.DataFrame({
    "n_peaks_EDA": peak_counts,
    "mean_peak_amp_EDA": mean_peak_amp,
    "std_peak_amp_EDA": std_peak_amp
}).values

c:\Users\Zita\Repositories\affective-states\venv\Lib\site-packages\numpy\_core\_methods.py:223: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\Zita\Repositories\affective-states\venv\Lib\site-packages\numpy\_core\_methods.py:181: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\Zita\Repositories\affective-states\venv\Lib\site-packages\numpy\_core\_methods.py:215: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


# Add extra labels to the data frame

In [17]:
# opening annotated data
with open(annotation_path, 'rb') as f:
    annotation_data = pickle.load(f)

# Inspect the type and structure
print(type(annotation_data))  # Check the data type
print(annotation_data.keys())

<class 'dict'>
dict_keys(['ar_seg', 'vl_seg', 'unc_seg', 'ts_seg'])


In [18]:
with open(questionnaire_path, 'rb') as f:
    quest_data = pickle.load(f)

print(type(quest_data))  # Check the data type
print(quest_data.keys())
print(len(quest_data["ID"]))

<class 'dict'>
dict_keys(['ID', 'device'])
1481


In [19]:
# Add the labels to the dataframe

df_physio[['ar_seg', 'vl_seg', 'unc_seg', 'ts_seg', 'ID', 'device']] = pd.DataFrame({
    "ar_seg": annotation_data["ar_seg"],
    "vl_seg": annotation_data["vl_seg"],
    "unc_seg": annotation_data["unc_seg"],
    "ts_seg": annotation_data["ts_seg"],
    "ID": quest_data["ID"],
    "device": quest_data["device"]
}).values

# Dealing with NaN values

In this section, I drop the collumn with heart rate data, which are removed by removing rows with more that 5 NaN values.

The following column with NaN values remain:
- 'EDA_quality_idx' - all values are integers between 5 and 1479 -> NaN values will be subsituted with -1 and dummy variable will be added
- 'PPG_quality_idx' - all values are floats between 0 and 1.480e+03 -> NaN values will be subsituted with -1 and dummy variable will be added
- 'mean_peak_amp_EDA' - all values are floats between 0 and 5.13961366e+02 -> NaN values will be subsituted with -1 and dummy variable will be added
- 'std_peak_amp_EDA' -  all values are floats between 0 and 2.50771934e+01 -> NaN values will be subsituted with -1 and dummy variable will be added
- 'unc_seg' - all the unique values are: 3.0 2.0 2.5 1.5 1.0 (this likely means high to low uncertainty of a segment.) -> NaN values will be subsituted with 4.0


In [20]:
#TODO: deal with NaN values in df_physio
df_physio_clean = df_physio.copy()

In [21]:
# check which columns are NaN
nan_columns = df_physio_clean.columns[df_physio.isna().any()].tolist()
print("Columns with NaN values: ", nan_columns)

Columns with NaN values:  ['EDA_quality_idx', 'PPG_quality_idx', 'mean_hr', 'std_hr', 'max_hr', 'min_hr', 'mean_hr_dot', 'std_hr_dot', 'max_hr_dot', 'min_hr_dot', 'mean_diff_hr_time', 'std_diff_hr_time', 'max_diff_hr_time', 'min_diff_hr_time', 'SDNN', 'rMSSD', 'mean_peak_amp_EDA', 'std_peak_amp_EDA', 'unc_seg']


In [22]:
# if a row contains more than 5 NaN values, drop it
df_physio_clean = df_physio_clean.dropna(thresh=len(df_physio_clean.columns) - 5)       # This drops thr rows without Heart Rate data
df_physio_clean = df_physio_clean.reset_index(drop=True)                                # reset the index after dropping rows

# The remaining NaN values are in the following columns:
nan_columns = df_physio_clean.columns[df_physio_clean.isna().any()].tolist()
print("Columns with NaN values: ", nan_columns)

Columns with NaN values:  ['EDA_quality_idx', 'PPG_quality_idx', 'mean_peak_amp_EDA', 'std_peak_amp_EDA', 'unc_seg']


In [23]:
# print unique values of a column
col_name = 'unc_seg'
print(df_physio_clean[col_name].unique())

[nan 3.0 2.0 2.5 1.5 1.0]


In [24]:
# in unc_seg, subsitute missing values with 4.0
df_physio_clean['unc_seg'] = df_physio_clean['unc_seg'].replace([np.nan], 4.0)

# in the columns with NaN values, substitute missing values -1.0
for col in nan_columns:
    df_physio_clean[col] = df_physio_clean[col].replace([np.nan], -1.0)

C:\Users\Zita\AppData\Local\Temp\ipykernel_8560\2923350185.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_physio_clean['unc_seg'] = df_physio_clean['unc_seg'].replace([np.nan], 4.0)


In [25]:
# Create two new dummy columns for d_quality and d_peak_EDA
df_physio_clean['d_quality'] = [0 if item == -1.0 else 1 for item in df_physio_clean['EDA_quality_idx']] # if there is nan in EDA quality then there is also no PPG quality
df_physio_clean['d_peak_EDA'] = [0 if item == -1.0 else 1 for item in df_physio_clean['mean_peak_amp_EDA']]

In [26]:
# compare lenghts of df_physio_clean and df_physio, reduction percentage
print("Length of df_physio_clean: ", len(df_physio_clean))
print("Length of df_physio: ", len(df_physio))
print("Reduction percentage: ", (len(df_physio) - len(df_physio_clean)) / len(df_physio) * 100)

Length of df_physio_clean:  1229
Length of df_physio:  1481
Reduction percentage:  17.01553004726536


# Save data in one dataframe
All column with lists are removed as they cannot be used as model input.


In [27]:
# Drop the columns that are of type array
df = df_physio_clean.drop(columns=["filt_EDA", "EDR", "hr", "ts", "hr_idx", "hr_time", "diff_hr_time", "successive_diff_hr_time", "filt_PPG", "sampling_rate", "packet_number", "raw_EDA", "raw_PPG", "filt_EDA_dot", "filt_EDA_ddot", "EDR_dot", "EDR_ddot", "hr_dot"])

In [28]:
df.describe(include='all')  # Display all columns in the DataFrame

,EDA_quality_idx,PPG_quality_idx,mean_filt_EDA,std_filt_EDA,max_filt_EDA,min_filt_EDA,mean_filt_EDA_dot,std_filt_EDA_dot,max_filt_EDA_dot,min_filt_EDA_dot,...,mean_peak_amp_EDA,std_peak_amp_EDA,ar_seg,vl_seg,unc_seg,ts_seg,ID,device,d_quality,d_peak_EDA
count,1229.000000,1229.000000,1.229000e+03,1229.000000,1229.000000,1229.000000,1229.000000,1229.000000,1229.000000,1229.000000,...,1229.000000,1229.000000,1229.0,1229.0,1229.000000,1229.0,1229.0,1229,1229.000000,1229.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,5.0,NaN,1057.0,162.0,11,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,3.0,NaN,5.0,28.0,34,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,373.0,461.0,NaN,11.0,69.0,146,NaN,NaN
mean,638.398698,580.759967,8.518683e+02,101.039673,1084.897840,712.410208,0.027107,0.637367,2.515501,-1.341855,...,792.325139,60.190670,NaN,NaN,3.639544,NaN,NaN,NaN,0.833198,0.792514
std,510.126441,536.428890,6.861064e+02,167.434512,845.786841,627.544339,0.228142,1.347412,5.334911,4.384717,...,768.081853,137.439592,NaN,NaN,0.831152,NaN,NaN,NaN,0.372951,0.405671
min,-1.000000,-1.000000,-7.359384e-33,0.000000,0.000000,-0.004448,-1.700899,0.000000,-0.212733,-42.403935,...,-1.000000,-1.000000,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000,0.000000
25%,141.000000,45.000000,3.168605e+02,15.579760,474.492613,165.045351,-0.035228,0.040408,0.099668,-0.722755,...,54.522609,0.000000,NaN,NaN,4.000000,NaN,NaN,NaN,1.000000,1.000000
50%,551.000000,410.000000,7.568800e+02,47.773928,949.004239,640.335167,0.000174,0.196589,0.683758,-0.202979,...,674.123587,8.152321,NaN,NaN,4.000000,NaN,NaN,NaN,1.000000,1.000000
75%,1141.000000,1164.000000,1.233927e+03,114.141120,1547.326922,1037.006491,0.077821,0.639126,2.432761,-0.063942,...,1213.845136,53.921937,NaN,NaN,4.000000,NaN,NaN,NaN,1.000000,1.000000


In [29]:
# Ensure the directory exists
output_path = os.path.join("c:/Users/Zita/Repositories/affective-states/data/processed", "processed_data.pkl")
df.to_pickle(output_path)
print("Data saved to pickle file successfully.")


Data saved to pickle file successfully.


## Add normalized dataset
I will apply 3 techniques:
1. Standardization (Z-Score Normalization, good for many ml models like Linear Regression, Logistic Regression, SVM, Neural Networks)
2. Min-Max Scaling (good for Algorithms that rely on distance metrics (e.g., KNN, SVM with RBF kernel))
3. Robust Scaling (best for data with heavy outliers)

In [40]:
col_to_normalize = [col for col in df.columns if col not in ["ar_seg", "vl_seg", "unc_seg", "ts_seg", "ID", "device"]]

# Standardization
scaler = StandardScaler()
df_standardized = df.copy()
df_standardized[col_to_normalize] = pd.DataFrame(scaler.fit_transform(df[col_to_normalize]), columns=col_to_normalize)

# Min-Max Scaling
scaler = MinMaxScaler()
df_min_max = df.copy()
df_min_max[col_to_normalize] = pd.DataFrame(scaler.fit_transform(df[col_to_normalize]), columns=col_to_normalize)

# Robust Scaling
scaler = RobustScaler()
df_robust = df.copy()
df_robust[col_to_normalize] = pd.DataFrame(scaler.fit_transform(df[col_to_normalize]), columns=col_to_normalize)

In [41]:
# Ensure the directory exists
output_path = os.path.join("c:/Users/Zita/Repositories/affective-states/data/processed", "standardized_data.pkl")
df_standardized.to_pickle(output_path)

output_path = os.path.join("c:/Users/Zita/Repositories/affective-states/data/processed", "min_max_data.pkl")
df_min_max.to_pickle(output_path)

output_path = os.path.join("c:/Users/Zita/Repositories/affective-states/data/processed", "robust_data.pkl")
df_robust.to_pickle(output_path)

print("All data saved to pickle file successfully.")


All data saved to pickle file successfully.
